# SVG(inf) for LQG

## Introduction
In this notebook we implement a simplified version of SVG($\infty$) for the LQG problem. Our intention is to show how one can learn a parameterized policy in LQG via Stochastic Value Gradients. Note that even if we successfully learn a policy, we do not necessarily know what components of the SVG framework were instrumental in doing so. The focus of the rest of our work will be on analyzing the tenets of this framework based on **gradient estimation quality** and **performance curvature approximation**.

### Imports

In [1]:
from __future__ import annotations
import logging
import itertools
import os.path as osp
from datetime import date
from numbers import Number
from typing import Optional

import pytorch_lightning as pl
import torch
import torch.nn as nn
import wandb
from ray.rllib import RolloutWorker
from ray.rllib import SampleBatch
from ray.rllib.evaluation.metrics import collect_episodes, summarize_episodes
from raylab.policy import TorchPolicy
from raylab.policy.modules.actor import DeterministicPolicy
from torch.optim import Optimizer
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import TensorDataset
from tqdm.auto import trange

import lqsvg.torch.named as nt
from lqsvg.envs import lqr
from lqsvg.envs.lqr.gym import TorchLQGMixin
from lqsvg.experiment.data import TrajectoryData
from lqsvg.experiment.models import MonteCarloSVG
from lqsvg.experiment.models import glorot_init_model
from lqsvg.experiment.policy import make_worker
from lqsvg.experiment.tqdm_util import collect_with_progress
from lqsvg.experiment.utils import group_batch_episodes
from lqsvg.experiment.utils import linear_feedback_distance
from lqsvg.experiment.utils import suppress_dataloader_warning
from lqsvg.policy.time_varying_linear import LQGPolicy

# https://github.com/PyTorchLightning/pytorch-lightning/issues/3431
logging.getLogger('lightning').setLevel(logging.WARNING)

---
## Algorithm

In [2]:
class Experiment:
    def __init__(self, run):
        self.run = run
        
        self.worker = None
        self.policy = None
        self.model = None
        self.artifact = None
        self.collect_metrics = None
        
    @property
    def dir(self) -> str:
        return self.run.dir
        
    def setup(self):
        self.worker = make_worker(env_config)
        rllib_policy = self.worker.get_policy()
        self.policy = rllib_policy.module.actor
        self.model = EnvModel(rllib_policy, run.config)
        
        self.artifact = self.create_artifact()
        self.collect_metrics = CollectMetrics()
        
    def create_artifact(self) -> wandb.Artifact:
        model = self.model.model
        return wandb.Artifact(f"svg_inf-lqg{model.n_state}.{model.n_ctrl}.{model.horizon}", type="model")

    def execute(self):
        """Main algorithm logic."""
        hparams = self.run.config
        dataset = self.create_dataset()
        optimizer = self.create_optimizer(self.policy)

        for itr in trange(hparams["iterations"], desc="SVG(inf)", unit="iteration"):
            if itr % 10 == 0:
                self.add_ckpt_to_artifact(itr)

            self.collect_trajs(self.worker, dataset)
            self.optimize_model(self.model, dataset)
            self.step_policy(self.policy, self.model, optimizer)

            # Logging
            self.log_iteration(itr)

        self.finish()
        
    def create_dataset(self) -> pl.LightningDataModule:
        return DataModule(self.run.config)
    
    def create_optimizer(self, policy: nn.Module) -> Optimizer:
        return torch.optim.Adam(policy.parameters(), lr=self.run.config["policy_lr"])
    
    def collect_trajs(self, worker: RolloutWorker, dataset: pl.LightningDataModule):
        dataset.collect_trajectories(worker, n_trajs=self.run.config["trajs_per_iter"])

    def optimize_model(self, model: pl.LightningModule, dataset: pl.LightningDataModule):
        hparams = self.run.config

        validation_results = ValidationResults()
        early_stopping = pl.callbacks.EarlyStopping(
            model.early_stop_on,
            min_delta=hparams["improvement_delta"],
            patience=hparams["patience"],
            mode="min",
            strict=True,
        )
        trainer = pl.Trainer(
            default_root_dir=self.dir,
            callbacks=[early_stopping, validation_results], 
            max_epochs=1000, 
            progress_bar_refresh_rate=0,  # don't show progress bar for model training
            weights_summary=None,  # don't print summary before training
            checkpoint_callback=False,  # don't save last model
        )
        with suppress_dataloader_warning():
            trainer.fit(model, datamodule=dataset)

        # Logging
        results = {
            "model_epochs": trainer.current_epoch + 1, 
            "model_nll": validation_results.last_validation_loss().item(),
        }
        self.run.log(results, commit=False)

    def step_policy(self, policy: DeterministicPolicy, model: pl.LightningModule, optimizer: Optimizer):
        svg = MonteCarloSVG(policy, model.model)
        optimizer.zero_grad()
        loss = -svg.value(self.run.config["svg_samples"])
        loss.backward()
        optimizer.step()
        
    def log_iteration(self, itr: int):
        pistar, _, _ = self.worker.env.solution
        logs = {
            "iteration": itr, 
            "distance_to_optimal": linear_feedback_distance(self.policy.standard_form(), pistar), 
            **self.collect_metrics(self.worker)
        }
        self.run.log(logs)
        
    def finish(self):
        self.add_ckpt_to_artifact(self.run.config["iterations"])            
        self.run.log_artifact(self.artifact)        

    def add_ckpt_to_artifact(self, itr: int):
        policy: TorchPolicy = self.worker.get_policy()
        path = osp.join(self.run.dir, f"module-iter={itr}.pt")
        torch.save(policy.module.state_dict(), path)
        self.artifact.add_file(path)

### Logging

In [3]:
class CollectMetrics:
    # Copied from https://github.com/ray-project/ray/blob/c409b5b63a6928e423428b700e528e35d791e8ea/rllib/execution/metric_ops.py#L47
    def __init__(self):
        self.episode_history = []
        self.to_be_collected = []

    def __call__(self, worker: RolloutWorker, min_history: int = 100, timeout_seconds: int = 180) -> dict:
        # Collect worker metrics.
        episodes, self.to_be_collected = collect_episodes(
            worker, to_be_collected=self.to_be_collected, timeout_seconds=timeout_seconds
        )
        orig_episodes = list(episodes)
        missing = min_history - len(episodes)
        if missing > 0:
            episodes.extend(self.episode_history[-missing:])
            assert len(episodes) <= min_history
        self.episode_history.extend(orig_episodes)
        self.episode_history = self.episode_history[-min_history:]
        return {k: v for k, v in summarize_episodes(episodes, orig_episodes).items() if isinstance(v, Number)}

In [4]:
class ValidationResults(pl.callbacks.Callback):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.saved_outputs = None

    def on_validation_epoch_start(self, trainer: pl.Trainer, pl_module: pl.LightningModule):
        self.saved_outputs: list[Tensor] = []
            
    def on_validation_batch_end(self, trainer: pl.Trainer, pl_module: pl.LightningModule, outputs: Tensor, *args, **kwargs):
        self.saved_outputs += [outputs]
        
    def last_validation_loss(self) -> Tensor:
        return torch.stack(self.saved_outputs, dim=0).mean(dim=0)

### Modules

In [5]:
class DataModule(pl.LightningDataModule):
    def __init__(self, hparams: dict):
        super().__init__()
        
        self.batch_size: float = hparams["dataset_batch_size"]
        self.train_val_split: tuple[float, float] = hparams["train_val_split"]

        self.full_dataset = None
        self.train_dataset, self.val_dataset = None, None
        self._itr_datasets: list[TensorDataset] = []
        
    def collect_trajectories(self, rollout_worker: RolloutWorker, n_trajs: int):
        worker = rollout_worker
        self._check_rollout_worker(worker)
        
        sample_batch = collect_with_progress(worker, n_trajs, prog=False)
        sample_batch = group_batch_episodes(sample_batch)
        trajs = sample_batch.split_by_episode()
        self._check_collected_trajs(trajs, worker, n_trajs)
        
        traj_dataset = TrajectoryData.trajectory_dataset(trajs)
        self._itr_datasets += [traj_dataset]
        self.full_dataset = ConcatDataset(self._itr_datasets)
        
    def setup(self, stage: Optional[str] = None):
        del stage
        train_frac, _ = self.train_val_split
        train_trajs = int(train_frac * len(self.full_dataset))
        val_trajs = len(self.full_dataset) - train_trajs

        self.train_dataset, self.val_dataset = random_split(self.full_dataset, (train_trajs, val_trajs))

    def train_dataloader(self) -> DataLoader:
        # pylint:disable=arguments-differ
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.batch_size)

    def val_dataloader(self) -> DataLoader:
        # pylint:disable=arguments-differ
        return DataLoader(self.val_dataset, shuffle=False, batch_size=self.batch_size)

    @staticmethod
    def _check_rollout_worker(worker: RolloutWorker):
        assert worker.rollout_fragment_length == worker.env.horizon * worker.num_envs
        assert worker.batch_mode == "truncate_episodes"
        
    @staticmethod
    def _check_collected_trajs(trajs: list[SampleBatch], worker: RolloutWorker, n_trajs: int):
        traj_counts = [t.count for t in trajs]
        assert all(c == worker.env.horizon for c in traj_counts), traj_counts
        total_ts = sum(t.count for t in trajs)
        assert total_ts == n_trajs * worker.env.horizon, total_ts        

In [6]:
class EnvModel(pl.LightningModule):
    early_stop_on: str = "val/loss"

    def __init__(self, policy: LQGPolicy, hparams: dict):
        super().__init__()
        self.model = policy.module.model

        self.hparams.learning_rate = hparams["model_lr"]
        glorot_init_model(self.model)

    def configure_optimizers(self):
        params = nn.ParameterList(
            itertools.chain(self.model.trans.parameters(), self.model.init.parameters())
        )
        optim = torch.optim.Adam(params, lr=self.hparams.learning_rate)
        return optim

    def forward(self, obs: Tensor, act: Tensor, new_obs: Tensor) -> Tensor:
        """Batched trajectory log prob."""
        # pylint:disable=arguments-differ
        return self.model.log_prob(obs, act, new_obs)

    def _compute_loss_on_batch(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> Tensor:
        del batch_idx
        obs, act, new_obs = (x.refine_names("B", "H", "R") for x in batch)
        return -self(obs, act, new_obs).mean()

    def training_step(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> Tensor:
        # pylint:disable=arguments-differ
        loss = self._compute_loss_on_batch(batch, batch_idx)
        self.log("train/loss", loss)
        return loss

    def validation_step(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> Tensor:
        # pylint:disable=arguments-differ
        loss = self._compute_loss_on_batch(batch, batch_idx)
        self.log("val/loss", loss)
        return loss

---
## Experiments

In [7]:
def calver() -> str:
    today = date.today()
    return f"{today.month}.{today.day}.0"

print("CalVer:", calver())

CalVer: 3.17.0


In [8]:
env_config = dict(n_state=8, n_ctrl=8, horizon=1000, num_envs=20)
hparams = {
    "iterations": 200,
    "trajs_per_iter": 20,
    "policy_lr": 3e-4,
    "improvement_delta": 0.0,
    "patience": 3,
    "svg_samples": 32,
    "dataset_batch_size": 32,
    "train_val_split": (0.8, 0.2),
    "model_lr": 1e-3,
    "env_config": env_config,
}

run = wandb.init(
    name="SVG(inf)",
    project="LQG-SVG",
    entity="angelovtt",
    tags=[calver()],
    mode="online",
    save_code=True,
    job_type="train",
)
run.config.update(hparams)

with run, nt.suppress_named_tensor_warning():
    experiment = Experiment(run)
    experiment.setup()
    experiment.execute()

wandb: Currently logged in as: angelovtt (use `wandb login --relogin` to force relogin)


SVG(inf):   0%|          | 0/200 [00:00<?, ?iteration/s]

2021-03-17 13:08:04,391	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


model_epochs,10
model_nll,639261.75
iteration,199
distance_to_optimal,13.90417
episode_reward_max,-61931.17195
episode_reward_min,-101351.81497
episode_reward_mean,-77556.79786
episode_len_mean,1000.0
episodes_this_iter,20
_runtime,1649
_timestamp,1615998930


model_epochs,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▂▂▂▂▂
model_nll,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇██
iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
distance_to_optimal,▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
episode_reward_max,▃▄▅▅▆▆▇▇▇▇▇█▇███▇▇▇██▇▇▇▇▆▆▅▄▄▅▄▄▄▄▂▃▁▁▁
episode_reward_min,▆▅▅▇▇▆█▇▇██▆▆▇▇▆▅▇▇▆▆▅▆▆▆▆▄▅▅▄▃▄▃▃▃▂▂▃▂▁
episode_reward_mean,▅▅▆▆▇▇▇▇▇███████▇▇▇▇▇▆▆▇▆▆▆▅▅▅▅▅▄▄▃▃▂▂▁▁
episode_len_mean,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episodes_this_iter,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
